# Trabalho Prático 1
## Computação Gráfica

Deiziane Natani da Silva  
2015121980  

Implementação do Flat shading, Gouraud Shading e Phong Shading.

In [2]:
import glfw
import pyrr
import numpy
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import OpenGL.GL.shaders


Shaders descrevem como processar dados no pipeline gráfico, ou seja, são um conjunto de instruções.

No código estão dois tipos de shader:

- **Vertex Shader**:
   - Como o nome indica, é o responsável pela manipulação dos vértices de um objeto tridimensional. O vertex shader processa cada ponto/vértice que você passa e retorna valores de posição para esse ponto.
   
   
- **Fragment Shader**:
   - Trabalha num pixel, ou um fragmento. É onde a profundidade, cor e todos os outros parâmetros dos gráficos são processados de acordo com as condições ambientais e de iluminação especificada


In [3]:
vertex_shader = '''

#version 330

uniform mat4 model_transform;
uniform mat4 camera;
uniform mat4 projection;
uniform vec3 camera_pos_input;

uniform float albedo_r;
uniform float albedo_g;
uniform float albedo_b;
uniform float ks;

in vec3 position;
in vec3 vertex_normal;

out vec3 view_position;
out vec3 pixel_position;
out vec3 vertex_color;
out vec3 camera_position;
out vec3 normal;
out vec3 light_source;
out vec4 ambient_light;

void main() {
    gl_Position = projection * camera * model_transform * vec4(position, 1.0);
    view_position = gl_Position.xyz;
    pixel_position = position;
    camera_position = camera_pos_input;
    
    if(vertex_normal == vec3(0.0, 0.0, 0.0)) {
        normal = normalize(position);
    }
    else {
        normal = normalize(vertex_normal);
    }
    
    light_source = vec3(5.0, 20.0, 10.0);
    float diffuse_red = albedo_r / 3.14 * dot(normal, light_source);
    float diffuse_green = albedo_g / 3.14 * dot(normal, light_source);
    float diffuse_blue = albedo_b / 3.14 * dot(normal, light_source);
    ambient_light = vec4(albedo_r/3.14, albedo_g/3.14, albedo_b/3.14, 0.0);

    vec3 reflection_vec = 2.0 * dot(light_source, normal) * normal - light_source;
    float specular = ks * dot(reflection_vec, normalize(camera_position));
    
    specular = max(0.0, specular);
    diffuse_red = max(0.0, diffuse_red);
    diffuse_green = max(0.0, diffuse_green);
    diffuse_blue = max(0.0, diffuse_blue);
    
    vertex_color = vec3(diffuse_red + specular, diffuse_green + specular, diffuse_blue + specular) + ambient_light.rgb;
}
'''




A **iluminação ambiente** (*ambient light*) é a luz que o objeto emite se não houver outras luzes.  

A **iluminação difusa** (*diffuse light*) é a luz que o objeto emite uniformemente em todas as direções a partir do ponto em que um raio de luz o atinge, coisas que têm uma refletância difusa alta seriam objetos com um acabamento fosco.

O parâmetro **Albedo** controla a cor base da superfície.  

In [4]:
fragment_shader = '''

#version 330

uniform float albedo_r;
uniform float albedo_g;
uniform float albedo_b;
uniform float ks;
uniform int shading;

in vec3 normal;
in vec3 view_position;
in vec3 pixel_position;
in vec3 vertex_color;
in vec3 camera_position;

in vec3 light_source;
in vec4 ambient_light;

out vec4 frag_color;

//0 - flat shading
//1 - gouraud shading
//2 - phong shading

void main() {  
    if(shading == 0) {
        vec3 x_tangent = dFdx(pixel_position);
        vec3 y_tangent = dFdy(pixel_position);
        vec3 face_normal = normalize(cross(x_tangent, y_tangent));
        
        float red_diffuse = albedo_r / 3.14 * dot(face_normal, light_source);
        float green_diffuse = albedo_g / 3.14 * dot(face_normal, light_source);
        float blue_diffuse = albedo_b / 3.14 * dot(face_normal, light_source);
        
        vec3 reflection_vec = 2.0 * dot(light_source, face_normal) * face_normal - light_source;
        float specular = ks * dot(reflection_vec, normalize(camera_position));
        
        specular = max(0.0, specular);
        red_diffuse = max(0.0, red_diffuse);
        green_diffuse = max(0.0, green_diffuse);
        blue_diffuse = max(0.0, blue_diffuse);
        
        frag_color = vec4(red_diffuse + specular, green_diffuse + specular, blue_diffuse + specular, 1.0) + ambient_light;
    }
    else if(shading == 1){
        frag_color = vec4(vertex_color, 1.0);
    }
    else {
        vec3 interpolated_normal = normalize(normal);
        
        float red_diffuse = albedo_r / 3.14 * dot(interpolated_normal, light_source);
        float green_diffuse = albedo_g / 3.14 * dot(interpolated_normal, light_source);
        float blue_diffuse = albedo_b / 3.14 * dot(interpolated_normal, light_source);
        
        vec3 reflection_vec = 2.0 * dot(light_source, interpolated_normal) * interpolated_normal - light_source;
        float specular = ks * dot(reflection_vec, normalize(camera_position));
        
        specular = max(0.0, specular);
        red_diffuse = max(0.0, red_diffuse);
        green_diffuse = max(0.0, green_diffuse);
        blue_diffuse = max(0.0, blue_diffuse);
        
        frag_color = vec4(red_diffuse + specular, green_diffuse + specular, blue_diffuse + specular, 1.0) + ambient_light;
    }
}
'''

O tipo mais simples de shading é o *__Flat Shading__*. Ele aplica o modelo de iluminação uma única vez para
cada polígono, gerando uma intensidade que é usada para sombrea-lo. Como resultado do Flat Shading, todos os vértices do polígono são coloridos com uma cor, permitindo a diferenciação entre polígonos adjacentes.

Para produzir um resultado mais suave, pode-se utilizar o *__Gouraud Shading__*. Ele ilumina ou colora diretamente cada vértice pela utilização da sua posição e da sua normal. 
No Gouraud Shading, como podemos ver, o maior trabalho é feito no Vertex Shader, enquanto no Fragment Shader passamos apenas a cor diretamente pelo pipeline.

O *__Phong Shading__* descreve como uma superfície reflete a luz, sendo uma combinação da reflexão difusa, da reflexão especular e da reflexão ambiente. Ao contrário do Gouraud Shading, a cor é calculada no Fragment Shader, ao invés do Vertex Shader.

In [ ]:
# This function gets called every time the window needs to be updated
# i.e. after being hidden by another window and brought back into view,
# or when the window's been resized.
# You should never call this directly, but use glutPostRedisply() to tell
# GLUT to do it instead.

def redraw():
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    gluSphere(quad, 2.0, 256, 256)
    glutSwapBuffers()
    
# GLUT calls this function when the windows is resized.
# All we do here is change the OpenGL viewport so it will always draw in the
# largest square that can fit the window

def resize(w, h):
    if h == 0:
        h = 1
    
    # ensuring our windows is a square
    if w > h:
        w = h
    else:
        h = w
        
    
    # reset the current viewport and perspective transformation
    glViewport(0, 0, w, h)
    
    # tell GLUT to call the redrawing function, in this case redraw()
    glutPostRedisplay()

# GLUT calls this function when a key is pressed. Here we just quit when ESC or
# 'q' is pressed.
def keyfunc(key, x, y):
    if key == 27 or key == 'q' or key == 'Q':
        exit(0)

#
# Sets up an OpenGL light.  This only needs to be called once
# and the light will be used during all renders.
#
def initLights():
    amb      = [ 1.0, 1.0, 1.0, 1.0 ]
    diff     = [ 1.0, 1.0, 1.0, 1.0 ]
    spec     = [ 1.0, 1.0, 1.0, 1.0 ]
    lightpos = [ 2.0, 2.0, 5.0, 1.0 ]

    glLightModelfv(GL_LIGHT_MODEL_AMBIENT, amb)
    glLightfv(GL_LIGHT0, GL_AMBIENT, amb)
    glLightfv(GL_LIGHT0, GL_DIFFUSE, diff)
    glLightfv(GL_LIGHT0, GL_SPECULAR, spec)
    glLightfv(GL_LIGHT0, GL_POSITION, lightpos)
    glEnable(GL_LIGHT0)
    
    # Turn on lighting.  You can turn it off with a similar call to
    # glDisable().
    glEnable(GL_LIGHTING);


#
# Sets the OpenGL material state.  This is remembered so we only need to
# do this once.  If you want to use different materials, you'd need to do this
# before every different one you wanted to use.
#

def initMaterial():
    emit = [0.0, 0.0, 0.0, 1.0]
    amb  = [0.0, 0.0, 0.0, 1.0]
    diff = [0.0, 0.0, 1.0, 1.0]
    spec = [1.0, 1.0, 1.0, 1.0]
    shiny = 20.0

    glMaterialfv(GL_FRONT, GL_AMBIENT, amb)
    glMaterialfv(GL_FRONT, GL_DIFFUSE, diff)
    glMaterialfv(GL_FRONT, GL_SPECULAR, spec)
    glMaterialfv(GL_FRONT, GL_EMISSION, emit)
    glMaterialfv(GL_FRONT, GL_SHININESS, shiny)


#
# Set up OpenGL state.  This does everything so when we draw we only need to
# actually draw the sphere, and OpenGL remembers all of our other settings.
#

def initGL():
    # Tell openGL to use Gouraud shading:
    glShadeModel(GL_FLAT)
    
    # Enable back-face culling:
    glEnable(GL_CULL_FACE)
    glCullFace(GL_BACK)

    # Enable depth-buffer test.
    glEnable(GL_DEPTH_TEST)
    
    # Set up projection and modelview matrices ("camera" settings) 
    # Look up these functions to see what they're doing.
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    glFrustum(-0.5, 0.5, -0.5, 0.5, 1, 10)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    gluLookAt(5, 5, 5, 0, 0, 0, 1, 0, 0)

    # set light parameters
    initLights()

    # set material parameters
    initMaterial()

    # initialize the "quadric" used by GLU to render high-level objects.
    global quad
    quad = gluNewQuadric()
    gluQuadricOrientation(quad, GLU_OUTSIDE)



def main():
    glutInit(sys.argv)
    glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE | GLUT_DEPTH)

    glutInitWindowSize(800, 800)
    glutInitWindowPosition(300, 100)

    glutCreateWindow("Shading")
    
    initGL()

    # set up GLUT callbacks.
    glutDisplayFunc(redraw)
    glutReshapeFunc(resize)
    glutKeyboardFunc(keyfunc)

    glutMainLoop()

    return 1
    
# run the script
if __name__ == "__main__":
    main()

Com o OpenGL, definimos vértices e então construímos todos os objetos que queremos desenhar na tela usando triângulos. Um VBO é um array que contém dados que irão para o seu GPU.

Create a green screen

Referências:  
https://medium.com/@yvanscher/an-introduction-to-shaders-in-opengl-c19a1376eda1   
http://www.cs.toronto.edu/~jacobson/phong-demo/  
https://en.wikipedia.org/wiki/Shading#Flat_vs_smooth_shading